Una empresa de telecomunicaciones está desplegando torres en una ciudad para mejorar la cobertura posible en una red móvil. Las torres que se instalen tienen un alcance limitado, y cada torre debe ubicarse estratégicamente para maximizar la cobertura, minimizando al mismo tiempo los costos de construcción.

Posibles datos de entrada:

· (xi, yi): son las coordenadas de cada una de las N torres que deben ser colocadas en un área urbana.

· R: Es el radio de cobertura de cada torre (o cualquier variable que represente el alcance de la señal)

· 𝑹𝒎𝒂𝒙: es el radio máximo que pueden tener las torres.

· 𝑵𝒎𝒂𝒙: es el número máximo de torres que puede haber

· (𝑿𝒎𝒊𝒏,𝒀𝒎𝒊𝒏,𝑿𝒎𝒂𝒙,𝒀𝒎𝒂𝒙): son las dimensiones del plano en donde pueden estar las torres.

· [({x0, y0, R, Costo}, {x1,y1, R2, Costo2})]: posible características de un individuo

Función objetivo:

· Maximizar la cobertura de la red móvil.

· Minimizar los costos de construcción y el solapamiento innecesario de torres.

· Una idea de como representar la función objetivo es:

𝐹(𝑥,𝑦,𝑅)=𝐶𝑜𝑏𝑒𝑟𝑡𝑢𝑟𝑎− 𝜆∗(𝐶𝑜𝑠𝑡𝑜+𝑠𝑜𝑙𝑎𝑝𝑎𝑚𝑖𝑒𝑛𝑡𝑜)

· Donde 𝝀 es un factor de penalización en los costos y solapamiento

Objetivos:

· Encontrar las mejores coordenadas (xi, yi) para cada torre

· Encontrar el mejor radio de cobertura R que maximice la cobertura total.

· Minimizar el costo asociado (Entre más cobertura tiene una torre, más cuesta)

Restricciones:

· Las torres tendrán un máximo de cobertura, donde cada una tendrá un Radio que no exceda ese límite.

· Hay un límite de cuantas torres puede haber.

· Las torres no pueden ser puestas fuera del plano, deben estar dentro del área definida.

· La superposición de coberturas no debe ser excesiva (los radios de las torres no deben coincidir más de un porcentaje).

In [9]:
import numpy as np

# Datos de entrada
Xmin, Ymin, Xmax, Ymax = 0, 0, 100, 100  # Dimensiones del plano
Rmax = 20  # Radio máximo de cobertura
Nmax = 10  # Número máximo de torres
costo_por_radio = 10  # Costo por unidad de radio
lambda_penalizacion = 0.5  # Factor de penalización
penalizacion_solapamiento = 0.1  # Penalización por solapamiento

# Parámetros del PSO
numero_particulas = 30  # Cantidad de partículas en el enjambre
max_iteraciones = 100  # Número máximo de iteraciones
inercia = 0.7  # Factor de inercia
c1, c2 = 1.5, 1.5  # Coeficientes de aceleración

# Inicializar partículas
posiciones_particulas = np.random.uniform(
    [Xmin, Ymin] * Nmax + [0] * Nmax,
    [Xmax, Ymax] * Nmax + [Rmax] * Nmax,
    (numero_particulas, 3 * Nmax)
)
velocidades_particulas = np.random.uniform(-1, 1, (numero_particulas, 3 * Nmax))
mejor_posicion_local = np.copy(posiciones_particulas)
mejor_posicion_global = None

# Mejor valor encontrado
mejor_valor_local = np.full(numero_particulas, np.inf)
mejor_valor_global = np.inf

# Función para calcular el solapamiento
def calcular_solapamiento(posiciones, radios):
    solapamiento = 0
    for i in range(len(posiciones)):
        for j in range(i + 1, len(posiciones)):
            distancia = np.linalg.norm(np.array(posiciones[i]) - np.array(posiciones[j]))
            if distancia < radios[i] + radios[j]:
                solapamiento += max(0, (radios[i] + radios[j] - distancia))
    return solapamiento

# Función objetivo
def funcion_objetivo(parametros):
    posiciones = parametros[:2 * Nmax].reshape((Nmax, 2))
    radios = parametros[2 * Nmax:]
    
    # Filtrar torres que no están en el plano
    en_limites = (posiciones[:, 0] >= Xmin) & (posiciones[:, 0] <= Xmax) & \
                 (posiciones[:, 1] >= Ymin) & (posiciones[:, 1] <= Ymax)
    posiciones = posiciones[en_limites]
    radios = radios[en_limites]
    
    # Calcular cobertura total
    cobertura = np.pi * np.sum(radios**2)
    
    # Calcular costo
    costo = np.sum(radios * costo_por_radio)
    
    # Penalización por solapamiento
    solapamiento = calcular_solapamiento(posiciones, radios)
    
    # Función objetivo
    return -(cobertura - lambda_penalizacion * (costo + solapamiento))

# PSO principal
for iteracion in range(max_iteraciones):
    print(f"\n--- Iteración {iteracion + 1}/{max_iteraciones} ---")
    for i in range(numero_particulas):
        # Evaluar la función objetivo
        valor = funcion_objetivo(posiciones_particulas[i])
        print(f"  Partícula {i + 1}: Valor función objetivo = {valor:.2f}")
        
        # Actualizar el mejor local
        if valor < mejor_valor_local[i]:
            print(f"    Mejor local actualizado para partícula {i + 1}")
            mejor_valor_local[i] = valor
            mejor_posicion_local[i] = posiciones_particulas[i]
        
        # Actualizar el mejor global
        if valor < mejor_valor_global:
            print(f"    Nuevo mejor global encontrado por partícula {i + 1}")
            mejor_valor_global = valor
            mejor_posicion_global = posiciones_particulas[i]
    
    # Actualizar velocidades y posiciones de las partículas
    for i in range(numero_particulas):
        componente_inercial = inercia * velocidades_particulas[i]
        componente_cognitivo = c1 * np.random.random() * (mejor_posicion_local[i] - posiciones_particulas[i])
        componente_social = c2 * np.random.random() * (mejor_posicion_global - posiciones_particulas[i])
        velocidades_particulas[i] = componente_inercial + componente_cognitivo + componente_social
        posiciones_particulas[i] += velocidades_particulas[i]
        
        # Restringir partículas a los límites
        posiciones_particulas[i] = np.clip(
            posiciones_particulas[i],
            [Xmin, Ymin] * Nmax + [0] * Nmax,
            [Xmax, Ymax] * Nmax + [Rmax] * Nmax
        )

    print(f"Mejor valor global hasta ahora: {-mejor_valor_global:.2f}")

# Resultados
mejores_posiciones = mejor_posicion_global[:2 * Nmax].reshape((Nmax, 2))
mejores_radios = mejor_posicion_global[2 * Nmax:]
posiciones_validas = (mejores_posiciones[:, 0] >= Xmin) & (mejores_posiciones[:, 0] <= Xmax) & \
                     (mejores_posiciones[:, 1] >= Ymin) & (mejores_posiciones[:, 1] <= Ymax)

mejores_posiciones = mejores_posiciones[posiciones_validas]
mejores_radios = mejores_radios[posiciones_validas]

print("\n--- Resultados finales ---")
print("Mejores coordenadas y radios:")
for i, (pos, radio) in enumerate(zip(mejores_posiciones, mejores_radios)):
    print(f"Torre {i + 1}: Posición = {pos}, Radio = {radio:.2f}")
print(f"Valor final de la función objetivo (cobertura - penalizaciones): {-mejor_valor_global:.2f}")



--- Iteración 1/100 ---
  Partícula 1: Valor función objetivo = -2544.61
    Mejor local actualizado para partícula 1
    Nuevo mejor global encontrado por partícula 1
  Partícula 2: Valor función objetivo = -2862.07
    Mejor local actualizado para partícula 2
    Nuevo mejor global encontrado por partícula 2
  Partícula 3: Valor función objetivo = -5967.33
    Mejor local actualizado para partícula 3
    Nuevo mejor global encontrado por partícula 3
  Partícula 4: Valor función objetivo = -5286.45
    Mejor local actualizado para partícula 4
  Partícula 5: Valor función objetivo = -4405.71
    Mejor local actualizado para partícula 5
  Partícula 6: Valor función objetivo = -3365.39
    Mejor local actualizado para partícula 6
  Partícula 7: Valor función objetivo = -2252.50
    Mejor local actualizado para partícula 7
  Partícula 8: Valor función objetivo = -3169.34
    Mejor local actualizado para partícula 8
  Partícula 9: Valor función objetivo = -2699.73
    Mejor local actualiz

  Partícula 26: Valor función objetivo = -9192.52
  Partícula 27: Valor función objetivo = -9231.75
  Partícula 28: Valor función objetivo = -9232.95
    Mejor local actualizado para partícula 28
  Partícula 29: Valor función objetivo = -9230.76
    Mejor local actualizado para partícula 29
  Partícula 30: Valor función objetivo = -9229.54
Mejor valor global hasta ahora: 9233.03

--- Iteración 29/100 ---
  Partícula 1: Valor función objetivo = -9231.72
    Mejor local actualizado para partícula 1
  Partícula 2: Valor función objetivo = -9224.29
  Partícula 3: Valor función objetivo = -9232.20
  Partícula 4: Valor función objetivo = -9227.35
    Mejor local actualizado para partícula 4
  Partícula 5: Valor función objetivo = -9219.86
  Partícula 6: Valor función objetivo = -9229.47
  Partícula 7: Valor función objetivo = -9228.21
  Partícula 8: Valor función objetivo = -9231.67
  Partícula 9: Valor función objetivo = -9232.38
  Partícula 10: Valor función objetivo = -9232.96
    Mejor l